# Lab4
## Overview
1. นำ Gazebo จาก Lab3 มาปรับแก้ไขให้สามารถควบคุมการเคลื่อนที่ของ joint แบบ Velocity Control ได้
2. ทำ Tracker เพื่อ feedback ค่า postion error จาก joint ที่ต้องการควบคุม และคำนวณค่าความเร็วที่ต้องให้กับ joint นั้นๆ

## 1. นำ Gazebo จาก Lab3 มาปรับแก้ไขให้สามารถควบคุมการเคลื่อนที่ของ joint แบบ Velocity Control ได้
ปรับแก้ config file ให้สามารถควบคุม joint แบบ velocity control ได้ โดยใช้คำสั่ง
```xml
controller_manager:
  ros__parameters:
    update_rate: 100

    joint_state_broadcaster:
      type: joint_state_broadcaster/JointStateBroadcaster
    
    forward_velocity_controller:
      type: velocity_controllers/JointGroupVelocityController

forward_velocity_controller:
  ros__parameters:
    joints:
      - joint_1
      - joint_2
      - joint_3
    command_interface:
      - velocity
    state_interface:
      - position
      - velocity
```

## 2. ทำ Tracker เพื่อ feedback ค่า postion error จาก joint ที่ต้องการควบคุม และคำนวณค่าความเร็วที่ต้องให้กับ joint นั้นๆ
subcriber จาก topic /joint_states และ /goal จาก topic /goal จะได้ค่า position ของ joint ที่ต้องการควบคุม และค่า position ที่ต้องการให้ได้ จากนั้นจึงคำนวณค่าความเร็วที่ต้องให้กับ joint นั้นๆ โดยใช้คำสั่ง
```python
    def doPID(self):

        for i in range(3):
            self.pos_error[i] = self.pos_goal[i] - self.pos_feedback[i]
            self.vel_control[i] = self.kp[i]*self.pos_error[i] + self.ki[i]*self.pos_error_sum[i] + self.kd[i]*self.pos_error_diff[i]
            self.pos_error_sum[i] += self.pos_error[i]*0.1
            self.pos_error_diff[i] = (self.pos_error[i] - self.pos_last_error[i])/0.1
            self.pos_last_error[i] = self.pos_error[i]

        self.state.data = False
        if self.pos_error[0] < 0.1 and self.pos_error[1] < 0.1 and self.pos_error[2] < 0.1:
            self.pos_error_sum = [0.0,0.0,0.0]
            self.pos_error_diff = [0.0,0.0,0.0]
            self.state.data = True
        
    def goal_callback(self, msg):
        self.pos_goal = msg.data

    def joint_callback(self, msg):

        self.pos_feedback = msg.position[0:3]
```

